In [ ]:
import requests as req
from urllib.parse import quote

In [ ]:
COURSE = 13984
ASSIGNMENT = 130963
ID = 87131
BASE_URL = "https://canvas.sydney.edu.au/api/v1"

In [ ]:
HEADERS = {
    "Authorization": "Bearer " + input("Token: "),
    "Content-Type": "multipart/form-data"
}

In [ ]:
def format_url(url):
    return url.replace("[COURSE]", str(COURSE)).replace("[ASSIGNMENT]", str(ASSIGNMENT)).replace("[ID]", str(ID))

def handle(response):
    if response.ok:
        return response.json()
    else:
        return f"{response.status_code}: {response.reason}"

def put(url, *args, **kwargs):
    print(HEADERS)
    print(BASE_URL + format_url(url))
    return handle(req.put(BASE_URL + format_url(url), *args, headers=HEADERS, **kwargs))

def get(url, *args, **kwargs):
    print(HEADERS)
    print(BASE_URL + format_url(url))
    return handle(req.get(BASE_URL + format_url(url), *args, headers=HEADERS, **kwargs))

In [ ]:
rubrics = get("/courses/[COURSE]/rubrics")
rubrics

In [ ]:
criteria = rubrics[0]['data']
criteria_ids = [[y['id'] for y in x['ratings']] for x in criteria]
criteria_ids = {
    'A': {
        'id': criteria[0]['id'],
        'ratings': criteria_ids[0]
    },
    'B': {
        'id': criteria[1]['id'],
        'ratings': criteria_ids[1]
    },
    'C': {
        'id': criteria[2]['id'],
        'ratings': criteria_ids[2]
    }
}

In [ ]:
criteria_ids

In [ ]:
def get_rating_id(crit, mark):
    for i, m in enumerate([85, 75, 65, 50, 0]):
        if mark > m:
            return criteria_ids[crit]['ratings'][i + 1]

def gen_mark(grade, comment, mark_a, mark_b, mark_c):
    r = ""
    if comment.strip():
        r += f"comment[text_comment]={quote(comment.strip())}"
    r += f"&submission[posted_grade]={grade}"
    r += f"&rubric_assessment[{criteria_ids['A']['id']}][points]={mark_a}"
    r += f"&rubric_assessment[{criteria_ids['B']['id']}][points]={mark_b}"
    r += f"&rubric_assessment[{criteria_ids['C']['id']}][points]={mark_c}"
    return r.strip('&')

#     r += f"&rubric_assessment[{criteria_ids['A']['id']}][rating_id]={get_rating_id('A', mark_a)}"
#     r += f"&rubric_assessment[{criteria_ids['B']['id']}][rating_id]={get_rating_id('B', mark_b)}"
#     r += f"&rubric_assessment[{criteria_ids['C']['id']}][rating_id]={get_rating_id('C', mark_c)}"

def mark_assignment(sid, *args):
    return put("/courses/[COURSE]/assignments/[ASSIGNMENT]/submissions/[ID]", gen_mark(*args))

In [ ]:
mark_assignment(470423053, 225, "You can put a comment here!", 55, 90, 25)

In [ ]:
gen_mark(225, "Excellent!\n\nDouble newlines? Also inbetween rubric marks...", 80, 70, 60).split('&')